In [1]:
# Import Packages

import googlemaps
import pandas as pd
from geopy.distance import geodesic
import time

In [1]:

def calculate_distance(origin, destination):
    return geodesic(origin, destination).miles

def extract_phone(place):
    return place.get('formatted_phone_number', '') or place.get('international_phone_number', '') or place.get('plus_code', '')

def extract_opening_hours(place):
    periods = place.get('opening_hours', {}).get('periods', [])
    if periods:
        return [period.get('open', {}).get('time') for period in periods]
    else:
        return place.get('opening_hours', {}).get('weekday_text', [])

def find_places(api_key, location, distance, place_types):
    gmaps = googlemaps.Client(key=api_key)

    # Geocoding to get latitude and longitude from the address
    geocode_result = gmaps.geocode(location)
    lat_lng = geocode_result[0]['geometry']['location']

    all_places = []

    for place_type in place_types:
        # Places API text search for each place type
        places_result = gmaps.places(
            query=place_type,
            location=lat_lng,
            radius=distance * 1609.34,  # Convert miles to meters
            open_now=False
        )

        places = []

        for place in places_result['results']:
            # Calculate the distance from the provided location in miles
            place_location = (place['geometry']['location']['lat'], place['geometry']['location']['lng'])
            distance_miles = calculate_distance((lat_lng['lat'], lat_lng['lng']), place_location)

            # Check if the distance is within the specified search distance
            if distance_miles <= distance:
                # Extract phone number information
                phone_number = extract_phone(place)

                # Extract opening hours information
                opening_hours = extract_opening_hours(place)

                # Extract the primary type (first element in the types array)
                primary_type = place.get('types', [])[0] if place.get('types', []) else ''

                # Extract website URL
                website_url = place.get('url', '')

                place_info = {
                    'Place_ID': place['place_id'],
                    'Name': place['name'],
                    'Address': place.get('formatted_address', ''),
                    'Type': primary_type,
                    'Distance': round(distance_miles, 2),
                    'Phone': phone_number,
                    'Website': website_url,
                    'Opening_Hours': opening_hours                    
                }

                places.append(place_info)

        # Print the count of each type for each object
        print(f"{place_type.capitalize()}: {len(places)}")

        # Extend the list of all places with the places for the current type
        all_places.extend(places)

    return all_places

# Prompt user for API key, location, distance, and place types
api_key = input("Enter your Google Maps API key: ")
location = input("Enter your location (address, city, etc.): ")
distance = float(input("Enter the search distance in miles: "))
place_types = input("Enter a comma-separated list of place types (e.g., clinic,nursing_home,day_program,adult_day_care): ").lower()

# Convert place types to a list
place_types_list = [place_type.strip() for place_type in place_types.split(',')]

# Find places based on user input
all_places = find_places(api_key, location, distance, place_types_list)

# Create DataFrame from the combined results
df = pd.DataFrame(all_places)

# Filter the DataFrame to keep only unique places based on 'Place_ID'
df_unique = df.drop_duplicates(subset='Place_ID')

# Sort the DataFrame by 'Distance' in ascending order
df_unique = df_unique[df_unique['Distance'] <= distance].sort_values(by='Distance')

# Save the unique DataFrame to an Excel file
output_file = 'MarketResearch.xlsx'
df_unique.to_excel(output_file, index=False)
print(f"Results saved to {output_file}")

# Display a few rows of the sorted DataFrame
display(df_unique.head())


ValueError: could not convert string to float: ''

In [3]:
# Read the Excel file into a DataFrame
df = pd.read_excel('MarketResearch.xlsx')

# Count rows and columns
num_rows, num_columns = df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


Number of rows: 76
Number of columns: 8
